https://towardsdatascience.com/deploying-a-simple-machine-learning-model-into-a-webapp-using-tensorflow-js-3609c297fb04

In [21]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import json
from random import shuffle

In [2]:
!nvidia-smi

Sun Nov 15 12:22:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.67       Driver Version: 451.67       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 105... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8    N/A /  N/A |     75MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

## Load in the dataset       

In [97]:
dataset = []
path = 'fft_data'
with open(path + '/ee_fft_data.txt', 'r') as myfile:
    data=myfile.read()

obj = json.loads(data)


for fft in obj:
   dataset.append((fft['vowel'], np.array(fft['FFT'], dtype=np.uint8)))


In [99]:
path = 'fft_data'
with open(path + '/ih_fft_data.txt', 'r') as myfile:
    data=myfile.read()

obj = json.loads(data)


for fft in obj:
   dataset.append((fft['vowel'], np.array(fft['FFT'], dtype=np.uint8)))

In [100]:
shuffle(dataset)

In [101]:
y_dataset = [vowel for vowel, fft in dataset]
x_dataset = [fft for vowel, fft in dataset]

In [102]:
# change to numpy arrays and reshape training data for tf
x_dataset = np.array(x_dataset)
x_dataset = x_dataset.reshape(len(x_dataset),512,1).astype('float32')

y_dataset = np.array(y_dataset).astype('float32')

In [103]:
# split in to train and test
cutoff = round(len(y_dataset)*0.8) # 80/20 split
X_train = x_dataset[:cutoff]
y_train = y_dataset[:cutoff]
X_test = x_dataset[cutoff:]
y_test = y_dataset[cutoff:]

In [104]:
len(x_dataset), len(X_train), len(x_test), len(X_train)+len(X_test)
len(y_dataset), len(y_train), len(y_test), len(y_train)+len(y_test)

(192, 154, 38, 192)

In [105]:
X_test[0]

array([[  0.],
       [ 56.],
       [130.],
       [155.],
       [146.],
       [132.],
       [158.],
       [157.],
       [120.],
       [120.],
       [127.],
       [ 99.],
       [ 79.],
       [ 97.],
       [ 81.],
       [ 34.],
       [  5.],
       [  1.],
       [ 72.],
       [101.],
       [100.],
       [ 69.],
       [ 12.],
       [ 15.],
       [ 21.],
       [ 10.],
       [ 29.],
       [ 40.],
       [ 29.],
       [  0.],
       [ 10.],
       [  0.],
       [ 22.],
       [ 15.],
       [  9.],
       [ 18.],
       [  6.],
       [  0.],
       [  0.],
       [ 11.],
       [  8.],
       [  0.],
       [  0.],
       [ 10.],
       [  0.],
       [  4.],
       [  0.],
       [ 42.],
       [ 83.],
       [ 87.],
       [ 41.],
       [ 85.],
       [ 98.],
       [ 77.],
       [ 93.],
       [111.],
       [ 94.],
       [ 16.],
       [ 54.],
       [ 46.],
       [ 33.],
       [ 76.],
       [ 77.],
       [ 18.],
       [ 83.],
       [ 96.],
       [ 7

## Create and Train the Model

In [106]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from keras.utils import np_utils

# flatten 28*28 images to a 784 vector for each image
#num_pixels = X_train.shape[1] * X_train.shape[2]


X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')


#X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
#X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
#y_train = np_utils.to_categorical(y_train)
#y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]





In [127]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
model = Sequential()
model.add(Dense(12, input_dim=512, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
16/16 [==============================] - 0s 812us/step - loss: 0.6675 - accuracy: 0.6169
Epoch 2/150
16/16 [==============================] - 0s 875us/step - loss: 0.6183 - accuracy: 0.6558
Epoch 3/150
16/16 [==============================] - 0s 1ms/step - loss: 0.5609 - accuracy: 0.6688
Epoch 4/150
16/16 [==============================] - 0s 915us/step - loss: 0.4997 - accuracy: 0.7078
Epoch 5/150
16/16 [==============================] - 0s 1ms/step - loss: 0.4368 - accuracy: 0.7273
Epoch 6/150
16/16 [==============================] - 0s 1ms/step - loss: 0.3786 - accuracy: 0.7857
Epoch 7/150
16/16 [==============================] - 0s 938us/step - loss: 0.3277 - accuracy: 0.9091
Epoch 8/150
16/16 [==============================] - 0s 1ms/step - loss: 0.2831 - accuracy: 0.9740
Epoch 9/150
16/16 [==============================] - 0s 938us/step - loss: 0.2443 - accuracy: 1.0000
Epoch 10/150
16/16 [==============================] - 0s 937us/step - loss: 0.2110 - accuracy: 1.00

In [128]:

# evaluate the keras model
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

5/5 [==============================] - 0s 793us/step - loss: 1.6116e-04 - accuracy: 1.0000
Accuracy: 100.00


In [131]:
predictions = model.predict(X_test)
predictions[:10]

array([[9.9987996e-01, 2.4953762e-05],
       [9.9999869e-01, 1.0827419e-07],
       [9.9986660e-01, 3.1765674e-05],
       [1.6348783e-06, 9.9999797e-01],
       [1.1570745e-04, 9.9995208e-01],
       [2.8888213e-05, 9.9997747e-01],
       [2.9806502e-05, 9.9998045e-01],
       [9.9944556e-01, 1.8560886e-04],
       [2.5016214e-05, 9.9998188e-01],
       [4.7727253e-06, 9.9999511e-01]], dtype=float32)

In [132]:
y_test[:10]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [134]:
model.save('fft_model.h5')

In [141]:
!tensorflowjs_converter --input_format keras fft_model.h5 ..\..\javascript\model_fft_js

2020-11-15 20:37:58.332109: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-11-15 20:37:58.332146: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [137]:
!ls

 Volume in drive C is Windows
 Volume Serial Number is 9497-8F11

 Directory of C:\Users\pedro\Documents\programming\python\tf_fft

11/15/2020  08:34 PM    <DIR>          .
11/15/2020  08:34 PM    <DIR>          ..
11/15/2020  01:26 PM    <DIR>          .ipynb_checkpoints
11/15/2020  01:42 PM    <DIR>          fft_data
11/15/2020  08:31 PM           106,328 fft_model.h5
11/15/2020  01:15 PM    <DIR>          js_model
11/15/2020  08:34 PM           287,004 keras fft.ipynb
11/15/2020  01:14 PM           768,088 model.h5
11/15/2020  01:09 PM                84 one-hot encoding key.txt
               4 File(s)      1,161,504 bytes
               5 Dir(s)  202,190,438,400 bytes free
